In [17]:
import json
import pandas as pd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

In [18]:
# Fonction pour lire les données de transcription
def read_transcription(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# Fonction pour lire les données du graphe de discours
def read_discourse_graph(file_path):
    with open(file_path, 'r') as file:
        data = [line.strip().split() for line in file]

    # Convertir les valeurs non numériques en indices numériques
    data = [(int(start), relation, int(end)) if start.isdigit() and end.isdigit() else (start, relation, end) for start, relation, end in data]

    return data

def create_dataframe(dialogue_id, transcription, discourse_graph, relation_dict, speaker_dict):
    rows = []

      # Iterate through all edges in the discourse graph
    for edge in discourse_graph:
        index_start, relation_type, index_end = edge

        # Retrieve speaker information
        speaker = transcription[index_start]['speaker']

        # Convert relation type to integer using the dictionary
        speaker_id = speaker_dict.get(speaker, -1)

        # Retrieve the sentence
        text = transcription[index_start]['text']

        # Convert relation type to integer using the dictionary
        relation_type_id = relation_dict.get(relation_type, -1)

        # Add a row to the DataFrame
        rows.append({
            'dialogue_id': dialogue_id,
            'index_start': index_start,
            'text': text,
            'index_end': index_end,
            'speaker_type': speaker_id,
            'speaker_text': speaker,
            'relation_type': relation_type_id,
            'relation_text': relation_type
        })

    # Create the DataFrame
    df = pd.DataFrame(rows)

    return df


# Fonction pour créer le dictionnaire de conversion des relations
def create_relation_dict(discourse_graph):
    relation_set = set()

    # Collecter toutes les relations uniques
    for edge in discourse_graph:
        relation_set.add(edge[1])

    # Créer un dictionnaire de conversion
    relation_dict = {relation: idx for idx, relation in enumerate(relation_set)}

    return relation_dict

# Fonction pour créer le dictionnaire de conversion des speakers
def create_speaker_dict(transcription):
    speaker_set = set()

    # Collecter tous les locuteurs uniques
    for utterance in transcription:
        speaker_set.add(utterance['speaker'])

    # Créer un dictionnaire de conversion
    speaker_dict = {speaker: idx for idx, speaker in enumerate(speaker_set)}

    return speaker_dict

def flatten(list_of_list):
    return [item for sublist in list_of_list for item in sublist]

# Function to get labels for a dialogue
def get_label(dialogue_id, index,labels_data):
    return labels_data.get(dialogue_id, [])[index]

In [19]:
# Remplacez 'votre_chemin' par le chemin correct
path_train= Path("data/training")
path_test= Path("data/test")

# Remplacez 'vos_dialogue_ids' par votre liste réelle d'identifiants de dialogue
dialogue_ids = ['ES2002', 'ES2005', 'ES2006', 'ES2007', 'ES2008', 'ES2009', 'ES2010', 'ES2012', 'ES2013', 'ES2015', 'ES2016', 'IS1000', 'IS1001', 'IS1002', 'IS1003', 'IS1004', 'IS1005', 'IS1006', 'IS1007', 'TS3005', 'TS3008', 'TS3009', 'TS3010', 'TS3011', 'TS3012']
dialogue_ids = flatten([[m_id+s_id for s_id in 'abcd'] for m_id in dialogue_ids])
dialogue_ids.remove('IS1002a')
dialogue_ids.remove('IS1005d')
dialogue_ids.remove('TS3012c')

dialogue_ids_test = ['ES2003', 'ES2004', 'ES2011', 'ES2014', 'IS1008', 'IS1009', 'TS3003', 'TS3004', 'TS3006', 'TS3007']
dialogue_ids_test = flatten([[m_id+s_id for s_id in 'abcd'] for m_id in dialogue_ids_test])

# Liste pour stocker les DataFrames de chaque dialogue
dfs = []
dfs_test = []

In [20]:
# Parcourir chaque dialogue
for dialogue_id in dialogue_ids:
    # Lire les données de transcription et de graphe de discours
    transcription = read_transcription(path_train / f'{dialogue_id}.json')
    discourse_graph = read_discourse_graph(path_train / f'{dialogue_id}.txt')
    
    # Créer le dictionnaire de conversion des relations
    relation_dict = create_relation_dict(discourse_graph)
    speaker_dict = create_speaker_dict(transcription)

    # Créer le DataFrame pour le dialogue actuel
    df = create_dataframe(dialogue_id, transcription, discourse_graph, relation_dict, speaker_dict)
    
    # Ajouter le DataFrame à la liste
    dfs.append(df)

    # Ajouter la dernière phrase avec NaN pour index_end et 'relation'
    last_utterance = transcription[-1]
    last_speaker = last_utterance['speaker']
    last_text = last_utterance['text']
    last_row = {
        'dialogue_id': dialogue_id,
        'index_start': len(transcription) - 1,
        'text': last_text,
        'index_end': np.nan,
        'speaker_type': speaker_dict.get(last_speaker, -1),
        'speaker_text': last_speaker,
        'relation_type': np.nan,
        'relation_text': np.nan
    }
    dfs.append(pd.DataFrame([last_row]))

# Parcourir chaque dialogue
for dialogue_id in dialogue_ids_test:
    # Lire les données de transcription et de graphe de discours
    transcription = read_transcription(path_test / f'{dialogue_id}.json')
    discourse_graph = read_discourse_graph(path_test / f'{dialogue_id}.txt')
    
    # Créer le dictionnaire de conversion des relations
    relation_dict = create_relation_dict(discourse_graph)
    speaker_dict = create_speaker_dict(transcription)

    # Créer le DataFrame pour le dialogue actuel
    df_test = create_dataframe(dialogue_id, transcription, discourse_graph, relation_dict, speaker_dict)
    
    # Ajouter le DataFrame à la liste
    dfs_test.append(df_test)

    # Ajouter la dernière phrase avec NaN pour index_end et 'relation'
    last_utterance = transcription[-1]
    last_speaker = last_utterance['speaker']
    last_text = last_utterance['text']
    last_row = {
        'dialogue_id': dialogue_id,
        'index_start': len(transcription) - 1,
        'text': last_text,
        'index_end': np.nan,
        'speaker_type': speaker_dict.get(last_speaker, -1),
        'speaker_text': last_speaker,
        'relation_type': np.nan,
        'relation_text': np.nan
    }
    dfs_test.append(pd.DataFrame([last_row]))

In [21]:
# Concaténer tous les DataFrames en un seul
df = pd.concat(dfs, ignore_index=True)
df_test = pd.concat(dfs_test, ignore_index=True)

with open("data/training_labels.json", 'r') as file:
    labels_data = json.load(file)

df['label'] = df.apply(lambda row: get_label(row['dialogue_id'], row['index_start'], labels_data), axis=1)

# Afficher le DataFrame final
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72623 entries, 0 to 72622
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   dialogue_id    72623 non-null  object 
 1   index_start    72623 non-null  int64  
 2   text           72623 non-null  object 
 3   index_end      72526 non-null  float64
 4   speaker_type   72623 non-null  int64  
 5   speaker_text   72623 non-null  object 
 6   relation_type  72526 non-null  float64
 7   relation_text  72526 non-null  object 
 8   label          72623 non-null  int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 5.0+ MB


,dialogue_id,index_start,text,index_end,speaker_type,speaker_text,relation_type,relation_text,label
0,ES2002a,0,Okay,1.0,3,PM,4.0,Continuation,0
1,ES2002a,1,Right,2.0,3,PM,4.0,Continuation,0
2,ES2002a,2,<vocalsound> Um well this is the kick-off meet...,3.0,3,PM,5.0,Explanation,1
3,ES2002a,3,Um <vocalsound> and um,4.0,3,PM,3.0,Elaboration,0
4,ES2002a,4,this is just what we're gonna be doing over th...,5.0,3,PM,4.0,Continuation,0


In [22]:
df['bert'] = df['text']
for transcription_id in dialogue_ids:
    bert_array = np.load('feature_bert/training/' + transcription_id + '.npy')
    
    # Obtenez les indices des lignes correspondant à la transcription_id
    indices = df[df['dialogue_id'] == transcription_id].index
    
    # Remplacez les valeurs de la colonne 'text' par les valeurs de bert_array
    for idx, value in enumerate(bert_array):
        df.at[indices[idx-1], 'bert'] = value

df_test['bert'] = df_test['text']
for transcription_id in dialogue_ids_test:
    bert_array_test = np.load('feature_bert/test/' + transcription_id + '.npy')
    
    # Obtenez les indices des lignes correspondant à la transcription_id
    indices = df_test[df_test['dialogue_id'] == transcription_id].index
    
    # Remplacez les valeurs de la colonne 'text' par les valeurs de bert_array
    for idx, value in enumerate(bert_array_test):
        df_test.at[indices[idx-1], 'bert'] = value


In [23]:
# Nombre d'éléments dans chaque liste
num_elements = len(df['bert'].iloc[0])

# Créez de nouvelles colonnes pour chaque élément dans la liste
new_columns = [f'coord_{i}' for i in range(num_elements)]

# Appliquez une fonction qui divise chaque liste en plusieurs colonnes
new_text_columns = df['bert'].apply(pd.Series)

# Renommez les nouvelles colonnes avec les noms spécifiques
new_text_columns.columns = new_columns

# Concaténez les nouvelles colonnes avec le DataFrame existant
df = pd.concat([df, new_text_columns], axis=1)



# Nombre d'éléments dans chaque liste
num_elements = len(df_test['bert'].iloc[0])

# Appliquez une fonction qui divise chaque liste en plusieurs colonnes
new_text_columns_test = df_test['bert'].apply(pd.Series)

# Renommez les nouvelles colonnes avec les noms spécifiques
new_text_columns_test.columns = new_columns

# Concaténez les nouvelles colonnes avec le DataFrame existant
df_test = pd.concat([df_test, new_text_columns_test], axis=1)


In [24]:
df.fillna(0)
df_test.fillna(0)

,dialogue_id,index_start,text,index_end,speaker_type,speaker_text,relation_type,relation_text,bert,coord_0,...,coord_374,coord_375,coord_376,coord_377,coord_378,coord_379,coord_380,coord_381,coord_382,coord_383
0,ES2003a,0,"Okay , well",1.0,3,PM,4.0,Continuation,"[-0.06684376, -0.10767134, 0.00158493, -0.0377...",-0.066844,...,0.089991,0.017770,0.004204,0.015277,-0.003793,0.035303,0.063118,-0.012957,0.057301,-0.023757
1,ES2003a,0,"Okay , well",2.0,3,PM,4.0,Continuation,"[-0.07298628, 0.052574955, -0.0014349254, -0.0...",-0.072986,...,0.084046,0.017392,-0.011899,0.002536,0.007794,0.037757,0.123899,0.037111,0.105005,0.098142
2,ES2003a,2,"Right ,",3.0,3,PM,1.0,Comment,"[-0.069116786, -0.030909952, 0.07359838, -0.06...",-0.069117,...,0.057107,-0.056366,0.004875,0.024899,-0.031351,-0.027805,0.029868,-0.065823,-0.027464,-0.058381
3,ES2003a,3,"my name's Adam Duguid ,",4.0,3,PM,5.0,Explanation,"[-0.08550309, -0.08060705, 0.04556774, 0.04994...",-0.085503,...,0.054002,0.058658,-0.060597,0.026250,-0.028345,0.014901,0.068747,0.050202,0.064570,0.009540
4,ES2003a,4,"we're here because of real reaction ,",5.0,3,PM,5.0,Explanation,"[-0.022576354, -0.028672846, -0.011893472, -0....",-0.022576,...,0.027556,0.016066,-0.029471,-0.008377,-0.026504,-0.055742,0.043690,-0.029209,0.006706,-0.045938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31021,TS3007d,1467,"Okay , um hereby is <disfmarker> the meeting i...",1468.0,3,ID,14.0,Acknowledgement,"[0.011373662, -0.05025587, -0.044846363, -0.06...",0.011374,...,-0.007787,0.059220,0.039703,0.081481,-0.015485,0.047069,-0.019950,0.070778,0.025126,-0.014484
31022,TS3007d,1468,<vocalsound> You declare . <vocalsound>,1469.0,0,ME,13.0,Clarification_question,"[-0.009817144, 0.014420933, 0.06607484, -0.006...",-0.009817,...,0.010156,0.032448,0.007771,0.055542,-0.037788,-0.018767,0.070564,0.056986,0.057005,-0.008631
31023,TS3007d,1469,I am the one who can say that . Yeah ?,1470.0,2,PM,4.0,Elaboration,"[-0.029034479, 0.004965499, 0.001875145, 0.015...",-0.029034,...,0.010273,0.058479,0.044487,-0.000141,-0.047491,0.009351,-0.023855,-0.010918,-0.039336,0.008305
31024,TS3007d,1467,"Okay , um hereby is <disfmarker> the meeting i...",1471.0,3,ID,14.0,Acknowledgement,"[-0.14641145, 0.005653019, -0.009107485, -0.00...",-0.146411,...,0.115167,-0.042565,0.024862,0.006545,0.066060,0.047295,0.097022,0.030912,0.091686,-0.034103


In [25]:
df['dif_start_end'] = df['index_end']-df['index_start']
df['word_count'] = df['text'].apply(lambda x: len(x.split(' ')))
df['nb_long_words'] = df['text'].apply(lambda x: sum(len(word) > 4 for word in x.split()))

In [26]:
df_test['dif_start_end'] = df_test['index_end']-df_test['index_start']
df_test['word_count'] = df_test['text'].apply(lambda x: len(x.split(' ')))
df_test['nb_long_words'] = df_test['text'].apply(lambda x: sum(len(word) > 4 for word in x.split()))

In [27]:
df.fillna(0)
df_test.fillna(0)

,dialogue_id,index_start,text,index_end,speaker_type,speaker_text,relation_type,relation_text,bert,coord_0,...,coord_377,coord_378,coord_379,coord_380,coord_381,coord_382,coord_383,dif_start_end,word_count,nb_long_words
0,ES2003a,0,"Okay , well",1.0,3,PM,4.0,Continuation,"[-0.06684376, -0.10767134, 0.00158493, -0.0377...",-0.066844,...,0.015277,-0.003793,0.035303,0.063118,-0.012957,0.057301,-0.023757,1.0,3,0
1,ES2003a,0,"Okay , well",2.0,3,PM,4.0,Continuation,"[-0.07298628, 0.052574955, -0.0014349254, -0.0...",-0.072986,...,0.002536,0.007794,0.037757,0.123899,0.037111,0.105005,0.098142,2.0,3,0
2,ES2003a,2,"Right ,",3.0,3,PM,1.0,Comment,"[-0.069116786, -0.030909952, 0.07359838, -0.06...",-0.069117,...,0.024899,-0.031351,-0.027805,0.029868,-0.065823,-0.027464,-0.058381,1.0,2,1
3,ES2003a,3,"my name's Adam Duguid ,",4.0,3,PM,5.0,Explanation,"[-0.08550309, -0.08060705, 0.04556774, 0.04994...",-0.085503,...,0.026250,-0.028345,0.014901,0.068747,0.050202,0.064570,0.009540,1.0,5,2
4,ES2003a,4,"we're here because of real reaction ,",5.0,3,PM,5.0,Explanation,"[-0.022576354, -0.028672846, -0.011893472, -0....",-0.022576,...,-0.008377,-0.026504,-0.055742,0.043690,-0.029209,0.006706,-0.045938,1.0,7,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31021,TS3007d,1467,"Okay , um hereby is <disfmarker> the meeting i...",1468.0,3,ID,14.0,Acknowledgement,"[0.011373662, -0.05025587, -0.044846363, -0.06...",0.011374,...,0.081481,-0.015485,0.047069,-0.019950,0.070778,0.025126,-0.014484,1.0,12,5
31022,TS3007d,1468,<vocalsound> You declare . <vocalsound>,1469.0,0,ME,13.0,Clarification_question,"[-0.009817144, 0.014420933, 0.06607484, -0.006...",-0.009817,...,0.055542,-0.037788,-0.018767,0.070564,0.056986,0.057005,-0.008631,1.0,5,3
31023,TS3007d,1469,I am the one who can say that . Yeah ?,1470.0,2,PM,4.0,Elaboration,"[-0.029034479, 0.004965499, 0.001875145, 0.015...",-0.029034,...,-0.000141,-0.047491,0.009351,-0.023855,-0.010918,-0.039336,0.008305,1.0,11,0
31024,TS3007d,1467,"Okay , um hereby is <disfmarker> the meeting i...",1471.0,3,ID,14.0,Acknowledgement,"[-0.14641145, 0.005653019, -0.009107485, -0.00...",-0.146411,...,0.006545,0.066060,0.047295,0.097022,0.030912,0.091686,-0.034103,4.0,12,5


In [28]:
X = df.drop(['dialogue_id','text','speaker_text','relation_text','bert','label'],axis=1)
y = df[['label']]

In [29]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from tqdm import tqdm
import pandas as pd

In [30]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Ajuster la forme des données
X_train = X_train.values.reshape(-1, len(X.columns))
X_test = X_test.values.reshape(-1, len(X.columns))

In [31]:
# Initialiser le modèle
model = KNeighborsClassifier()

#Train the model
model.fit(X_train, y_train)

#Predictions
y_pred = model.predict(X_test)

# Évaluer le modèle
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("\nRésultats du modèle :")
print("Accuracy: ", accuracy)
print("F1-score: ", f1)

# Matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred)

conf_df = pd.DataFrame(conf_matrix, index=['Actual 0', 'Actual 1'], columns=['Predicted 0', 'Predicted 1'])

print("\nMatrice de confusion :")
print(conf_df)

ValueError: Input X contains NaN.
KNeighborsClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
X_train = df.drop(['dialogue_id','text','speaker_text','relation_text','bert','label'],axis=1)
y_train = df[['label']]
X_test = df_test.drop(['dialogue_id','text','speaker_text','relation_text','bert'],axis=1)
X_train = X_train.values.reshape(-1, X_train.shape[1])
X_test = X_test.values.reshape(-1, X_test.shape[1])

In [ ]:
clf = KNeighborsClassifier()

# Ajuster le modèle
clf.fit(X_train, y_train)



DecisionTreeClassifier()

In [ ]:
test_pred = clf.predict(X_test)
print(np.sum(test_pred))
print(np.sum(y_test))

NameError: name 'clf' is not defined

In [ ]:
# Créer un dictionnaire pour stocker les prédictions
test_labels_final = {dialogue_id: [] for dialogue_id in dialogue_ids_test}

# Parcourir les lignes de df_test
for dialogue_id in dialogue_ids_test:
    # Obtenez les indices des lignes correspondant au dialogue_id
    indices = df_test[df_test['dialogue_id'] == dialogue_id].index
    # Ajouter les valeurs de test_label[index] au dictionnaire
    test_labels_final[dialogue_id] = test_pred[indices].tolist()


In [ ]:
with open("test_labels_text_knc.json", "w") as file:
    json.dump(test_labels_final, file, indent=4)